# DFViewer

The DFViewerWidget allows you to display a dataframe like the the full BuckarooWidget, without any extra UI options.

This works particularly well for embedding Buckaroo into widget or dashboarding frameworks like Solara

In [ ]:
import pandas as pd
from buckaroo import DFViewer

# Using the default DFViewer is straightforward

In [ ]:
df = pd.DataFrame({'a':[10, 20, 339, 887], 'b': ['foo', 'bar', None, 'baz']})
DFViewer(df)

## Removing pinned_rows

In [ ]:
DFViewer(df, pinned_rows=[])

## It works with polars too

In [ ]:
import polars as pl
from buckaroo.polars_buckaroo import PolarsDFViewer
pl_df = pl.DataFrame({'a':[10, 20, 339, 887], 'b': ['foo', 'bar', None, 'baz']})
PolarsDFViewer(pl_df)

In [ ]:
from buckaroo.buckaroo_widget import RawDFViewerWidget, BuckarooWidget
from buckaroo.dataflow.widget_extension_utils import (configure_buckaroo)
from buckaroo.dataflow.dataflow_extras import Sampling

def DFViewer(df,
             column_config_overrides=None,
             extra_pinned_rows=None, pinned_rows=None,
             extra_analysis_klasses=None, analysis_klasses=None,
             widget_klass=BuckarooWidget):
    """
    Display a DataFrame with buckaroo styling and analysis, no extra UI pieces

    column_config_overrides allows targetted specific overriding of styling

    extra_pinned_rows adds pinned_rows of summary stats
    pinned_rows replaces the default pinned rows

    extra_analysis_klasses adds an analysis_klass
    analysis_klasses replaces default analysis_klass
    """
    BuckarooKls = configure_buckaroo(
        widget_klass,
        extra_pinned_rows=extra_pinned_rows, pinned_rows=pinned_rows,
        extra_analysis_klasses=extra_analysis_klasses, analysis_klasses=analysis_klasses)

    bw = BuckarooKls(df, column_config_overrides=column_config_overrides)
    dfv_config = bw.df_display_args['dfviewer_special']['df_viewer_config']
    df_data = bw.df_data_dict['main']
    summary_stats_data = bw.df_data_dict['all_stats']
    return RawDFViewerWidget(
        df_data=df_data, df_viewer_config=dfv_config, summary_stats_data=summary_stats_data)

df = pd.DataFrame({'a':[10, 20, 339, 887], 'b': ['foo', 'bar', None, 'baz']})
#DFViewer(df)

class HeadTailSample(Sampling):
    pre_limit = 5
    max_columns = 1
    serialize_limit = 11
    head_tail_size = 5
    
    @classmethod
    def serialize_sample(kls, df):
        if kls.serialize_limit and len(df) > kls.serialize_limit:
            
            sampled = df.sample(kls.serialize_limit)
            if isinstance(sampled, pd.DataFrame):
                return sampled.sort_index()
            return sampled
        return df


class TwoBuckaroo(BuckarooWidget):
    sampling_klass = TwoSample
DFViewer(df, widget_klass=TwoBuckaroo)


In [ ]:
from buckaroo.buckaroo_widget import RawDFViewerWidget, BuckarooWidget
from buckaroo.dataflow.widget_extension_utils import (configure_buckaroo)
from buckaroo.dataflow.dataflow_extras import Sampling
import numpy as np
N = 100
df = pd.DataFrame({'a':  np.random.lognormal(25, .3, N), 'b':[2]*N})
#DFViewer(df)

class HeadTailSample(Sampling):
    pre_limit = False # disables any sampling for summary stats
    serialize_limit = 11
    head_tail_size = 5
    
    @classmethod
    def serialize_sample(kls, df):
        if kls.serialize_limit and len(df) > kls.serialize_limit:
            if isinstance(df, pd.DataFrame):
                ret_df = pd.concat([df[:kls.head_tail_size], df[(-1*kls.head_tail_size):]])
                return ret_df
        return df

class HeadTailBuckaroo(BuckarooWidget):
    sampling_klass = HeadTailSample
DFViewer(df, widget_klass=HeadTailBuckaroo)